Повторный парсинг не прогрузившихся объявлений

In [1]:
#Загружаем нужные библиотеки

import pandas as pd

from time import sleep

from tqdm import tqdm_notebook

from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.common.exceptions import NoSuchElementException

import os
import re

In [ ]:
# вытаскиваем названия файлов в список file_list

path_avito = 'C:/00_Projects/Rus_mushrooms/Parsing_results/Avito/'
file_list = []


for root, dirs, files in os.walk(path_avito):
    
    for filename in files:
        file_list += [filename]      

In [ ]:
%%time

# преобразовываю и объединяю в один датафрейм

mushrooms_avito = pd.DataFrame()

for i in file_list:
    temp = pd.read_excel(path_avito + i)
    
    mushrooms_avito = mushrooms_avito.append(temp, ignore_index = True)

In [ ]:
mushrooms_avito.shape

In [ ]:
mushrooms_avito.sample(3)

### Повторный парсинг ###

Собираю незагрузившиеся объявления второй раз.

In [ ]:
# нахожу урлы объявлений

temp = mushrooms_avito[
    (mushrooms_avito.topic == 'нет') |
    (mushrooms_avito.city == 'нет') |
    (mushrooms_avito.ads_text == 'нет')
]

In [ ]:
temp.shape

In [ ]:
# удаляю из рабочего датафрейма строки с значением "нет"

mushrooms_avito = mushrooms_avito[~(
    (mushrooms_avito.topic == 'нет') |
    (mushrooms_avito.city == 'нет') |
    (mushrooms_avito.ads_text == 'нет')
)]

In [ ]:
mushrooms_avito.shape

In [ ]:
# записываю в список ссылки

links = temp.url.to_list()

In [ ]:
len(links)

In [ ]:
# запускаю Хром

driver = Chrome('C:/work/chromedriver/chromedriver.exe')

In [ ]:
%%time

# список для данных
data = []


for url in tqdm_notebook(links):
    
    driver.get(url)
    sleep(5)

   
    try:
        topic = driver.find_element_by_class_name('title-info-title-text').text
        
    except:
        topic = 'нет'
        
    
    try:
        seller = driver.find_element_by_class_name('seller-info-name').text
        
    except:
        seller = 'нет'
        
    
    try:
        city = driver.find_element_by_class_name('item-address__string').text
    
    except:
        city = 'нет'
    
    
    try:
        ads_text = driver.find_element_by_class_name('item-description-text').text
        
    except:
        ads_text = 'нет'
            
    try:
        price = driver.find_element_by_xpath('/html/body/div[3]/div[1]/div[3]/div[4]/div[2]/div[1]/div/div[1]/div/div[1]/div/div/span/span[1]').text
        
    except:
        price = 'нет'
        
    try:
        user_url = driver.find_element_by_class_name('seller-info-name.js-seller-info-name').find_element_by_tag_name('a').get_attribute('href')
    
    except:
        user_url = 'нет'
    
    
    data.append([topic, city, ads_text, price, seller, user_url, url])  
    

In [ ]:
# преобразовываем в датафрейм

head = ['topic', 'city', 'ads_text', 'price', 'seller', 'user_id', 'url']
new = pd.DataFrame(data, columns=head)

In [ ]:
new.shape

In [ ]:
# удаляю дубликаты

new = new.drop_duplicates()

In [ ]:
new.shape

In [ ]:
new.sample(3)

In [ ]:
# удаляю объявления, которые не загрузились второй раз
# это объявления, которые уже удалены или сняты с публикации

new = new[~(
    (new.topic == 'нет') |
    (new.city == 'нет')
)]

In [ ]:
new.shape

In [ ]:
# добавляю данные к основному фрейму

mushrooms_avito = mushrooms_avito.append(new, ignore_index=True)

In [ ]:
mushrooms_avito.shape

### Удаляю дубликаты ###

In [ ]:
# создаю тестовый датафрейм

test = mushrooms_avito[mushrooms_avito.duplicated(keep=False)]

In [ ]:
# нахожу индексы повторябщихся строк

print(test.groupby(test.columns.tolist()).apply(lambda x: x.index.tolist()).values.tolist())

In [ ]:
# смотрю, что за строки

mushrooms_avito.loc[[6729, 9152]]

In [ ]:
# смотрю, что за строки

mushrooms_avito.loc[[5482, 10200]]

In [ ]:
mushrooms_avito.shape

In [ ]:
mushrooms_avito = mushrooms_avito.drop_duplicates()

In [ ]:
mushrooms_avito.shape

In [ ]:
# записываю в файл

mushrooms_avito.to_excel('C:/00_Projects/Rus_mushrooms/Avito.xlsx', index=False, encoding='utf-8')